In [3]:
import pandas as pd

df_scrape = pd.read_sql_table('resultdb_Stormfront', 'sqlite:////home/sam/far-right/stormfront/resultdb.sqlite')

In [4]:
print(df_scrape.shape)

df_scrape.head()

(97291, 4)


,taskid,url,result,updatetime
0,5c40eb08be70f35854ffafb8664ccc32,https://www.stormfront.org/forum/t390760,"{""url"": ""https://www.stormfront.org/forum/t390...",1.482262e+09
1,6fccf8f76446722cbd9b960788d7907f,https://www.stormfront.org/forum/t543403,"{""url"": ""https://www.stormfront.org/forum/t543...",1.482262e+09
2,125c26b84f5042c68d84218ddbdf8114,https://www.stormfront.org/forum/t1176587,"{""url"": ""https://www.stormfront.org/forum/t117...",1.482262e+09
3,332b1f09517e3d141ef100b0d94b9c47,https://www.stormfront.org/forum/t1186708,"{""url"": ""https://www.stormfront.org/forum/t118...",1.482262e+09
4,56a7a39456d9f4b837f3b81473f101f4,https://www.stormfront.org/forum/t1190477,"{""url"": ""https://www.stormfront.org/forum/t119...",1.482262e+09


Each page of threads has a row in this table. We must unpack the `result` json blob to the individual posts.

Each `result` blob contains several keys:

* The `username` key is a mistake and can be ignored.
* The `url` key also encodes pagination information for a thread. Page 13 of a thread will appear as `<thread_url>-13/`. This will need to be extracted out of the url.

In [5]:
import json

thread_page = json.loads(df_scrape.result[1000])
print("Result keys: {}\nUrl: {}".format("\t".join(thread_page.keys()), thread_page['url']))

Result keys: title	posts	username	url
Url: https://www.stormfront.org/forum/t1185584-13/


In [6]:
def extract_page(url):
    base, _, page = url.partition('-')
    
    if page:
        return base.strip('/'), int(page.strip('/'))
    else:
        return base.strip('/'), 1

extract_page('https://www.stormfront.org/forum/t1185584-13/'), extract_page('https://www.stormfront.org/forum/t1185584/')

(('https://www.stormfront.org/forum/t1185584', 13),
 ('https://www.stormfront.org/forum/t1185584', 1))

In [7]:
dict_list_scrape = []

for i, row in df_scrape.iterrows():
    page = json.loads(row.result)
    
    dict_list_scrape.append(page)
    
    if i % 300 == 0:
        print("Progress: {}".format(i))

Progress: 0
Progress: 300
Progress: 600
Progress: 900
Progress: 1200
Progress: 1500
Progress: 1800
Progress: 2100
Progress: 2400
Progress: 2700
Progress: 3000
Progress: 3300
Progress: 3600
Progress: 3900
Progress: 4200
Progress: 4500
Progress: 4800
Progress: 5100
Progress: 5400
Progress: 5700
Progress: 6000
Progress: 6300
Progress: 6600
Progress: 6900
Progress: 7200
Progress: 7500
Progress: 7800
Progress: 8100
Progress: 8400
Progress: 8700
Progress: 9000
Progress: 9300
Progress: 9600
Progress: 9900
Progress: 10200
Progress: 10500
Progress: 10800
Progress: 11100
Progress: 11400
Progress: 11700
Progress: 12000
Progress: 12300
Progress: 12600
Progress: 12900
Progress: 13200
Progress: 13500
Progress: 13800
Progress: 14100
Progress: 14400
Progress: 14700
Progress: 15000
Progress: 15300
Progress: 15600
Progress: 15900
Progress: 16200
Progress: 16500
Progress: 16800
Progress: 17100
Progress: 17400
Progress: 17700
Progress: 18000
Progress: 18300
Progress: 18600
Progress: 18900
Progress: 19200


In [8]:
for i, row in df_scrape.iterrows():
    thread, page_number = extract_page(dict_list_scrape[i]['url'])
    df_scrape.loc[i, 'thread_url'] = thread
    df_scrape.loc[i, 'page_number'] = page_number
    
    if i % 1000 == 0:
        print("Progress: {}".format(i))

Progress: 0
Progress: 1000
Progress: 2000
Progress: 3000
Progress: 4000
Progress: 5000
Progress: 6000
Progress: 7000
Progress: 8000
Progress: 9000
Progress: 10000
Progress: 11000
Progress: 12000
Progress: 13000
Progress: 14000
Progress: 15000
Progress: 16000
Progress: 17000
Progress: 18000
Progress: 19000
Progress: 20000
Progress: 21000
Progress: 22000
Progress: 23000
Progress: 24000
Progress: 25000
Progress: 26000
Progress: 27000
Progress: 28000
Progress: 29000
Progress: 30000
Progress: 31000
Progress: 32000
Progress: 33000
Progress: 34000
Progress: 35000
Progress: 36000
Progress: 37000
Progress: 38000
Progress: 39000
Progress: 40000
Progress: 41000
Progress: 42000
Progress: 43000
Progress: 44000
Progress: 45000
Progress: 46000
Progress: 47000
Progress: 48000
Progress: 49000
Progress: 50000
Progress: 51000
Progress: 52000
Progress: 53000
Progress: 54000
Progress: 55000
Progress: 56000
Progress: 57000
Progress: 58000
Progress: 59000
Progress: 60000
Progress: 61000
Progress: 62000
Progr

In [9]:
posts = []
for i, thread_page in enumerate(dict_list_scrape):
    for post_number, post in enumerate(thread_page['posts']):
        thread_url, page_number = extract_page(thread_page['url'])
        post['thread_url'] = thread_url
        post['page_number'] = page_number
        post['post_in_thread'] = (page_number - 1)*10 + post_number + 1
        
        posts.append(post)
    if i % 1000 == 0:
        print('.', end='')

..................................................................................................

In [22]:
df_posts = pd.DataFrame(posts)

In [12]:
print(df_posts.shape)

df_posts.head()

(972846, 10)


,author,author_join_date,author_location,author_posts,author_title,content,date,page_number,post_in_thread,thread_url
0,salvete amici,Oct 2005,"Christian, Free-market Libertarian","3,602",Forum Member,"<div id=""post_message_4230580"">&#13;\n\t\t\t&#...","05-24-2007, 03:19 PM",1,1,https://www.stormfront.org/forum/t390760
1,Royal Blue,Oct 2005,,"1,813",Forum Member,"<div id=""post_message_4230591"">&#13;\n\t\t\t&#...","05-24-2007, 03:27 PM",1,2,https://www.stormfront.org/forum/t390760
2,ArcticThunder,May 2006,U.S.A.,"3,940",Account Inactive,"<div id=""post_message_4230635"">&#13;\n\t\t\t&#...","05-24-2007, 03:52 PM",1,3,https://www.stormfront.org/forum/t390760
3,BeautynBrains1488,May 2007,Texico Amexica Where Wetbacks Rule,654,Forum Member,"<div id=""post_message_4230666"">&#13;\n\t\t\t&#...","05-24-2007, 04:13 PM",1,4,https://www.stormfront.org/forum/t390760
4,LOKI-Nevada,Mar 2002,Fled to Utah,"7,697",Account Disabled by Request,"<div id=""post_message_4230668"">&#13;\n\t\t\t&#...","05-24-2007, 04:16 PM",1,5,https://www.stormfront.org/forum/t390760


In [23]:
df_posts.count()

author              972846
author_join_date    972846
author_location     972846
author_posts        972846
author_title        972846
content             972846
date                972846
page_number         972846
post_in_thread      972846
thread_url          972846
dtype: int64

Yay! We've created a dataframe of posts. Save.

In [59]:
df_posts.to_csv('/home/sam/far-right/stormfront/posts.csv', index=False)

# Clean/normalize fields!

### First, the easy stuff...

* Remove commas from `author_posts` and make that field an integer
* Transform `author_join_date` into a known format, like 'YYYY-MM'

In [25]:
df_posts['author_posts'] = df_posts.author_posts.str.replace(',' ,'')

df_posts.author_posts.head()

0    3602
1    1813
2    3940
3     654
4    7697
Name: author_posts, dtype: object

In [26]:
import numpy as np
df_posts.author_posts.replace('n/a', np.nan, inplace=True)

author              972846
author_join_date    972846
author_location     972846
author_posts        961380
author_title        972846
content             972846
date                972846
page_number         972846
post_in_thread      972846
thread_url          972846
dtype: int64

In [35]:
df_posts[pd.to_numeric(df_posts.author_posts).isnull()].author_title.value_counts()

Guest    11466
Name: author_title, dtype: int64

In [36]:
df_posts['author_posts'] = pd.to_numeric(df_posts.author_posts)

In [37]:
from dateutil import parser

def parse_date(date_string):
    if date_string:
        parsed = parser.parse(date_string)
        return parsed
        
parse_date('Jun 2004'), parse_date('')

(datetime.datetime(2004, 6, 4, 0, 0), None)

In [41]:
df_posts['author_join_date'] = df_posts['author_join_date'].apply(parse_date)

### Slightly tricker

* The `date` field somtimes contains "Yesterday, [time of day]", or "Today, [time of day]". Correlate that post with its entry in the `df_threads` Dataframe to get the exact scrape time.

In [115]:
df_posts[df_posts.date.str.startswith('Yesterday')].head()

,author,author_join_date,author_location,author_posts,author_title,content,date,page_number,post_in_thread,thread_url
80,Pagonis,2016-05-22,,133.0,Forum Member,"<div id=""post_message_13839855"">&#13;\n\t\t\t&...","Yesterday, 01:16 PM",1,1,https://www.stormfront.org/forum/t1191985
81,WhiteNationalist,2012-11-22,The United Banana Republic,19938.0,under the Black Sun,"<div id=""post_message_13839887"">&#13;\n\t\t\t&...","Yesterday, 01:38 PM",1,2,https://www.stormfront.org/forum/t1191985
82,OLYMPUS,2014-08-22,Greece,13726.0,"""Friend of Stormfront""Sustaining Member","<div id=""post_message_13840103"">&#13;\n\t\t\t&...","Yesterday, 04:12 PM",1,3,https://www.stormfront.org/forum/t1191985
83,Pagonis,2016-05-22,,133.0,Forum Member,"<div id=""post_message_13840143"">&#13;\n\t\t\t&...","Yesterday, 04:33 PM",1,4,https://www.stormfront.org/forum/t1191985
84,OLYMPUS,2014-08-22,Greece,13726.0,"""Friend of Stormfront""Sustaining Member","<div id=""post_message_13840150"">&#13;\n\t\t\t&...","Yesterday, 04:36 PM",1,5,https://www.stormfront.org/forum/t1191985


In [42]:
import datetime

def parse_date_sometimes_relative(date_string, thread_url, page_number):
    try:
        return parser.parse(date_string)
    except TypeError:
        scrape_date = datetime.datetime.fromtimestamp(df_scrape[(df_scrape.thread_url == thread_url) & (df_scrape.page_number == page_number)].updatetime.iloc[0])
        day, time_of_day_string = date_string.split(',')
        time_of_day = parser.parse(time_of_day_string)
        if day.startswith('Today'):
            return datetime.datetime(scrape_date.year, scrape_date.month, scrape_date.day,
                                     time_of_day.hour, time_of_day.minute, time_of_day.second)
        
        if day.startswith('Yesterday'):
            return datetime.datetime(scrape_date.year, scrape_date.month, scrape_date.day - 1,
                         time_of_day.hour, time_of_day.minute, time_of_day.second)
print(parse_date_sometimes_relative('05-24-2007, 03:19 PM', None, None))

# Scraped on December 20th
print(parse_date_sometimes_relative('Today, 09:28 AM', 'https://www.stormfront.org/forum/t1191985', 1))
print(parse_date_sometimes_relative('Yesterday, 01:16 PM', 'https://www.stormfront.org/forum/t1191985', 1))

2007-05-24 15:19:00
2016-12-20 09:28:00
2016-12-19 13:16:00


In [ ]:
for i, row in df_posts.iterrows():
    if i % 1000 == 0:
        print('.', end='')
    df_posts.loc[i, 'post_date'] = parse_date_sometimes_relative(row['date'], row['thread_url'], row['page_number'])

..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [45]:
df_posts.count()

author              972846
author_join_date    961380
author_location     972846
author_posts        961380
author_title        972846
content             972846
date                972846
page_number         972846
post_in_thread      972846
thread_url          972846
post_date           972846
dtype: int64

In [46]:
df_posts.drop(['date', 'page_number'], axis=1).to_csv('/home/sam/data/stormfront/posts.csv', index=False)

In [47]:
df_posts.drop('date', axis=1, inplace=True)

# Extract states from `author_location` :)

* Tokenize the author_location field
* If any given token matches either the full name of a state, or the abbreviation for a state, then mark that entry as belonging to that state.
* Circle back on any record with multiple entries.

In [139]:
import requests

r = requests.get('http://www.fonz.net/blog/wp-content/uploads/2008/04/states.csv')

In [218]:
states = set()
state_abbreviations = set()
abbrevs_to_states = dict()

for line in r.text.split('\n')[1:-1]:
    state, abbrev = line.split(',')
    state = state.strip('"')
    abbrev = abbrev.strip('"')
    
    states.add(state.lower())
    state_abbreviations.add(abbrev.upper())
    if (abbrev.upper() != 'OR' and abbrev.upper() != 'IN'):
        state_abbreviations.add(abbrev.lower())
        state_abbreviations.add(abbrev.capitalize())
        
    abbrevs_to_states[abbrev.upper()] = state.lower()

In [159]:
df_authors = df_posts.groupby('author')['author_join_date', 'author_location', 'author_posts', 'author_title'].max().reset_index()

df_authors.head()

,author,author_join_date,author_location,author_posts,author_title
0,**JT**,2001-09-22,Western Washington,726.0,Forum Member
1,*Hither_Shore*,2004-09-22,,452.0,Forum Member
2,*turtle_88*,2008-01-22,,74.0,Account Inactive
3,+213Theodoric+,2004-04-22,"Thunderbolt, Colonial Georgia, Norð Amerika",49.0,Forum Member
4,+dezelman+,2006-06-22,,52.0,Forum Member


In [167]:
import nltk
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

In [169]:
locations = list(df_authors.author_location.unique())

In [220]:
def token_is_state(t):
    if t.lower() in states:
        return t.lower()
    if t in state_abbreviations:
        return abbrevs_to_states[t.upper()]
    
print(token_is_state('or'))
print(token_is_state('OR'))
print(token_is_state('ar'))
print(token_is_state('Fl'))
print(token_is_state('Alaska'))

None
oregon
arkansas
florida
alaska


In [221]:
from collections import defaultdict
locations_to_states = defaultdict(list)

for location in locations:
    tokens = [x for x in tokenizer.tokenize(location.replace('.', ''))]
    for i, t in enumerate(tokens):
        
        # Northeast, not Nebraska
        try:
            if t == 'NE' and token_is_state(tokens[i+1]):
                continue 
        except IndexError:
            pass
        
        state = token_is_state(t)
        if state:
            locations_to_states[location].append(state)

    if 'district of columbia' in locations_to_states[location] and 'washington' in locations_to_states[location]:
        locations_to_states[location].remove('washington')
    
    if 'california' in locations_to_states[location] and 'louisiana' in locations_to_states[location]:
        locations_to_states[location].remove('louisiana')
    
    if locations_to_states[location]:
        locations_to_states[location] = locations_to_states[location][0]

In [222]:
loc_dict = {x: y or None for x, y in locations_to_states.items()}

In [223]:
df_authors['primary_state'] = df_authors.author_location.apply(loc_dict.get)

In [226]:
df_authors.tail()

,author,author_join_date,author_location,author_posts,author_title,primary_state
15443,Иисус Христос,2006-09-22,,95.0,Account Inactive,None
15444,МΔLΙCΕ СΘDΕ,2007-09-22,Virginia,25.0,Forum Member,virginia
15445,Наследие,2007-10-22,,1473.0,Account Inactive,None
15446,Уберменшен,2007-03-22,Русь,219.0,Forum Member,None
15447,Шhіte Đ≡vіl,2006-09-22,S.C,129.0,Forum Member,south carolina


In [227]:
df_authors.to_csv('/home/sam/far-right/stormfront/authors.csv', index=False)

# Create thread table that has thread title

In [232]:
def extract_title(title_string):
    return title_string.partition(' - Page ')[0].partition(' - Stormfront')[0]

print(extract_title('Trump wins White women by a landslide! - Page 12 - Stormfront'))
print(extract_title('Lamentations of Michelle Obama - Stormfront'))

Trump wins White women by a landslide!
Lamentations of Michelle Obama


In [236]:
for i, row in df_scrape.iterrows():
    title = extract_title(dict_list_scrape[i]['title'])
    df_scrape.loc[i, 'title'] = title
    if i % 1000 == 0:
        print('.', end='')

..................................................................................................

In [237]:
df_scrape.head()

,taskid,url,result,updatetime,thread_url,page_number,title
0,5c40eb08be70f35854ffafb8664ccc32,https://www.stormfront.org/forum/t390760,"{""url"": ""https://www.stormfront.org/forum/t390...",1.482262e+09,https://www.stormfront.org/forum/t390760,1.0,Cat-Sized African Rats Plague Florida
1,6fccf8f76446722cbd9b960788d7907f,https://www.stormfront.org/forum/t543403,"{""url"": ""https://www.stormfront.org/forum/t543...",1.482262e+09,https://www.stormfront.org/forum/t543403,1.0,"EXPERT predicts food riots, tent cities, tax r..."
2,125c26b84f5042c68d84218ddbdf8114,https://www.stormfront.org/forum/t1176587,"{""url"": ""https://www.stormfront.org/forum/t117...",1.482262e+09,https://www.stormfront.org/forum/t1176587,1.0,"China About To Trigger ""The Super Crash"""
3,332b1f09517e3d141ef100b0d94b9c47,https://www.stormfront.org/forum/t1186708,"{""url"": ""https://www.stormfront.org/forum/t118...",1.482262e+09,https://www.stormfront.org/forum/t1186708,1.0,People magazine names Dwayne Johnson as the se...
4,56a7a39456d9f4b837f3b81473f101f4,https://www.stormfront.org/forum/t1190477,"{""url"": ""https://www.stormfront.org/forum/t119...",1.482262e+09,https://www.stormfront.org/forum/t1190477,1.0,Special snowflakes in Massachusetts


In [239]:
df_scrape.groupby('thread_url').title.first().reset_index().to_csv('/home/sam/far-right/stormfront/thread-titles.csv', index=False)